In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,4,5,6,7'
from models.scMCP import scMCP
import anndata as ad

In [ ]:
adata =  sc.read('.h5ad')

In [ ]:
coexpress_threshold = 0.1
from trainer.co_express_networks import get_similarity_network,GeneSimNetwork
num_similar_genes_co_express_graph = [20]
gene_list = list(adata.var.index)
data_path = 'embeddings/'
train_gene_set_size = 17911
node_map = {x: it for it, x in enumerate(gene_list)}
seed = 2025
edge_list = get_similarity_network(network_type = 'go', adata = adata, 
                                   threshold = coexpress_threshold, 
                                   k = num_similar_genes_co_express_graph, gene_list = gene_list, data_path = data_path, data_name = dataset_name, split = split, seed =seed, train_gene_set_size = train_gene_set_size)
sim_network = GeneSimNetwork(edge_list, gene_list, node_map = node_map)
model = scMCP(
            num_genes=2048,
             uncertainty=False,
             num_gnn_layers=None,
             decoder_hidden_size=None,
             num_gene_gnn_layers=None,
             input_genes_ens_ids=None,
             scfm_genes_ens_ids=None,
            drug_dim=768,
    drug_gene_attr =False,
                 hidden_size = 128,
    coexpress_network = sim_network,
             pos_emb_graph = None,
                 grn_node2vec_file='/home/MBDAI206AA201/jupyter/yhz/sc/scdata/GeneCompass-main/downstream_tasks/PRCEdrug/GraphEmbedding/node2vec/emb_grn/grn_emb_total.pkl',
                 ppi_node2vec_file='/home/MBDAI206AA201/jupyter/yhz/sc/scdata/GeneCompass-main/downstream_tasks/PRCEdrug/GraphEmbedding/node2vec/emb_ppi/ppi_emb_total.pkl',
                 model_type = 'ppi_grn_mode')

In [ ]:
import torch.nn as nn
model = nn.DataParallel(model, device_ids=[i for i in range(torch.cuda.device_count())])  
model.module.load_state_dict(torch.load('./checkpoint/load_best_epoch_all.pt')['model_state_dict'])


In [ ]:
sci_adata= test_data[(test_data.obs['cell_type'] == 'T')&(test_data.obs.dose==0.0)]

In [ ]:
sampled_data=sci_adata.to_df().sample(frac=1, random_state=2025)
control_obs_df = pd.DataFrame(index=sampled_data.index)
for i in sampled_data.index.to_list():
    cell_line = i
    control_obs_df.loc[cell_line,'Drug'] = 'DMSO'
    control_obs_df.loc[cell_line,'dose'] = 0.0
    control_obs_df.loc[cell_line,'SMILES'] = 'unknown'
    control_obs_df.loc[cell_line,'cell_type'] = str(cell_line)
    control_obs_df.loc[cell_line,'dose'] = 0.0
    control_obs_df.loc[cell_line,'cov_drug_name'] = str(cell_line)+'_DMSO'
    control_obs_df.loc[cell_line,'cov_drug_dose_name'] = str(cell_line)+'_DMSO_0.0'
control_obs_df
import anndata as ad
adata_control = ad.AnnData(sampled_data,obs=control_obs_df,var=adata.var)
adata_control

In [ ]:
drug_df = pd.read_csv('./drug/L4200-FDA-Drugs.csv')

In [ ]:
from tqdm import tqdm


dose_list = [1000]

adata_obs_df = pd.DataFrame()

for cell_type in tqdm(control_obs_df.index.to_list()):
    adata_obs_df_ = pd.DataFrame()
    temp_drug_df = drug_df.copy()
    temp_drug_df['cell_type'] = cell_type  
    
    for dose in dose_list:  
        temp = temp_drug_df.copy()
        temp['dose'] = dose 
        adata_obs_df_ = pd.concat([adata_obs_df_, temp], ignore_index=True)
    
    adata_obs_df = pd.concat([adata_obs_df, adata_obs_df_], ignore_index=True)

adata_obs_df['Drug'] = adata_obs_df['drug']
adata_obs_df['cov_drug_name'] = adata_obs_df['cell_type'].astype(str) + '_' + adata_obs_df['Drug'].astype(str)
adata_obs_df['cov_drug_dose_name'] = adata_obs_df['cell_type'].astype(str) + '_' + \
                                    adata_obs_df['Drug'].astype(str) + '_' + \
                                    adata_obs_df['dose'].astype(str)

import anndata as ad

In [ ]:
adata_drug_X = np.zeros((adata_obs_df.shape[0], 17911))

adata_test = ad.AnnData(adata_drug_X,obs=adata_obs_df,var=adata_control.var)
adata_obs_all = pd.concat([adata_control.obs, adata_test.obs], ignore_index=True)
adata_obs_all.index = adata_obs_all.index.astype(str)
adata_X_all = np.concatenate((adata_control.X, adata_test.X),axis=0)

adata_drug = ad.AnnData(adata_X_all,obs=adata_obs_all,var=adata_control.var)
adata_drug.obs.loc[adata_drug.obs.Drug!='DMSO', 'kidney_split']='test'
adata_drug.obs.loc[adata_drug.obs.Drug=='DMSO', 'kidney_split']=''
adata_drug.obs.cell_type = adata_drug.obs.cell_type.astype(str)

In [ ]:
for cell_type in tqdm(adata_drug.obs.cell_type.unique().tolist()):
    adata_drug.obs.loc[(adata_drug.obs.Drug!='DMSO') & (adata_drug.obs.cell_type==str(cell_type)), 'paired_control_index']=adata_drug.obs[(adata_drug.obs.Drug=='DMSO') & (adata_drug.obs.cell_type==str(cell_type))].index.values[0]
adata_drug.obs

In [ ]:
adata_drug.obs['cov_drug_dose'] = (
    adata_drug.obs['cell_type'].astype(str).str.cat(
        adata_drug.obs['drug'].astype(str), 
        sep='_'
    ).str.cat(
        adata_drug.obs['dose'].astype(str), 
        sep='_'
    )
)

In [ ]:
from data._utils import Condition_encoder, Drug_SMILES_encode, rank_genes_groups_by_cov, Drug_dose_encoder,load_smiles_bedding_drug_dose_encoder,load_smiles_from_path_embedding_drug_dose_encoder
from scipy import sparse
from torch.utils.data import Dataset
class DrugDoseAnnTokenDataset(Dataset):

    def __init__(self,
                 adata,
                 dtype='train',
                 obs_key='cov_drug',
                 comb_num=1,
                 split_key='random_split_0',
                 top_gene_per_cell=2048 ,
                  smiles_dataset='sciplex',
                 cell_key='cell_type'

                 ):
        self.dtype = dtype
        self.obs_key = obs_key        
        self.top_gene_per_cell = top_gene_per_cell 

        if sparse.issparse(adata.X):
            self.dense_adata = sc.AnnData(
                X=adata.X.A, 
                obs=adata.obs.copy(deep=True),
                var=adata.var.copy(deep=True)
            )
        else:
            self.dense_adata = adata

        self.all_gene_ids = self.dense_adata.var.index.tolist() 
        self.total_gene_num = len(self.all_gene_ids)  

        if self.total_gene_num < self.top_gene_per_cell:
            self.top_gene_per_cell = self.total_gene_num
            
        self.drug_adata = self.dense_adata[self.dense_adata.obs['dose'] != 0.0]
        self.data = torch.tensor(self.drug_adata.X, dtype=torch.float32)
        self.dense_data = torch.tensor(self.dense_adata.X, dtype=torch.float32)
        
        self.paired_control_index = self.drug_adata.obs['paired_control_index'].tolist()
        self.dense_adata_index = self.dense_adata.obs.index.to_list()  
        self.drug_type_list = self.drug_adata.obs['SMILES'].to_list()
        self.dose_list = self.drug_adata.obs['dose'].to_list()
        self.obs_list = self.drug_adata.obs[obs_key].to_list()
        self.celltype_list = self.drug_adata.obs[cell_key].to_list()
        if smiles_dataset!='sciplex' and smiles_dataset!='tagoe':
            self.encode_drug_doses = load_smiles_from_path_embedding_drug_dose_encoder(drug_SMILES_list=self.drug_type_list,file_path=smiles_dataset, dose_list=self.dose_list)
        else:
            self.encode_drug_doses = load_smiles_bedding_drug_dose_encoder(drug_SMILES_list=self.drug_type_list,dataset=smiles_dataset, dose_list=self.dose_list)
        self.encode_drug_doses = torch.tensor(self.encode_drug_doses, dtype=torch.float32)
    def __len__(self):
        return len(self.drug_adata)  


    def __getitem__(self, index):
        outputs = dict()
        control_idx = self.dense_adata_index.index(str(self.paired_control_index[index]))
        control_expr = self.dense_data[control_idx, :]  
        top_vals, top_indices = torch.topk(control_expr, self.top_gene_per_cell, dim=0)
        outputs['x_top_vals'] = self.data[index]  
        outputs['x_top_indices'] = top_indices 
        outputs['control_top_vals'] = top_vals
        top_gene_tokens = []
        top_gene_ids = [] 
        for idx in top_indices:
            gene_id = self.all_gene_ids[idx.item()]  
            top_gene_ids.append(gene_id)
            top_gene_tokens.append(id_token.get(name2id.get(gene_id), 0)) 

        outputs['top_gene_tokens'] = torch.tensor(top_gene_tokens, dtype=torch.long)
        outputs['top_gene_ids'] = top_gene_ids 

        outputs['drug_dose'] = self.encode_drug_doses[index, :]
        outputs['label'] = outputs['drug_dose']
        outputs['cov_drug'] = self.obs_list[index]
        outputs['data'] = self.dense_adata[index, :] 
        
        return {
            'features': (outputs['control_top_vals'], outputs['x_top_vals']),
            'label': outputs['label'],
            'cov_drug': outputs['cov_drug'],
            'top_gene_tokens': outputs['top_gene_tokens'],
            'top_gene_indices': outputs['x_top_indices'],  
            'top_gene_ids': outputs['top_gene_ids'],
            'control_raw': control_expr,
            'celltype':self.celltype_list[index]
        }
test_dataset = DrugDoseAnnTokenDataset(adata_drug,smiles_dataset='./embeddings/FDA_smiles_embeddings.pkl',  obs_key='cov_drug_dose', comb_num=1,split_key='random_split0',cell_key='cell_type')


In [ ]:
from torch.distributions import Bernoulli, Normal
from data.Dataset import DrugDoseAnnTokenDataset
import pickle
file = open('human_mouse_tokens.pickle', 'rb')
id_token = pickle.load(file)

In [ ]:

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=400, shuffle= True,num_workers=8)

In [ ]:
from torch.distributions import Bernoulli, Normal
def sample_ziln(zi_prob, log_normal_dist, device=None, truncate_max=None):

    if device is None:
        device = zi_prob.device
    bernoulli_dist = Bernoulli(probs=zi_prob)
    zero_mask = bernoulli_dist.sample().to(device)  
    
    y_samples = log_normal_dist.sample()  
    if truncate_max is not None:
        y_samples = torch.clamp(y_samples, max=truncate_max)
    

    samples = y_samples * (1 - zero_mask) 
    return samples

In [ ]:
import functools
import pickle
device='cuda'
file = open('Gene_id_name_dict.pickle', 'rb')
gene = pickle.load(file)
file.close()
name2id = {value:key for key,value in gene.items()}
from torch.autograd import Variable, grad
from torch.nn import functional as F
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tqdm
import torch
import torch.nn as nn
from scipy.spatial.distance import cityblock,euclidean

def make_noise(batch_size, shape, volatile=False):
    tensor = torch.randn(batch_size, shape)
    noise = Variable(tensor, volatile)
    noise = noise.to("cuda", dtype=torch.float32)
    return noise
from torch.distributions import NegativeBinomial, normal

In [ ]:
import tqdm
all_samples = []
control_list_samples = []
target_list_samples = []
celltype_list = []
from torch.distributions import Categorical, Normal, MixtureSameFamily, Independent
with torch.no_grad():
    for iteration in range(1): 
        batch_samples = []
        control_list = []
        target_list = []
        cov_drug_list = []
        emb_list = [] 

        raw_control_list = []
        raw_target_list = []
        gene_index_list = []
        gene_token_list = []
        loop_v = tqdm.tqdm(enumerate(test_dataloader), total =len(test_dataloader))
        for j, vdata in loop_v:
            (control, target) = vdata['features']
            encode_label = vdata['label']
            gene_token = vdata['top_gene_tokens']
            gene_token_list.append(gene_token)
            control = control.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            encode_label = encode_label.to(device, dtype=torch.float32)
            b_size = control.size(0)
            cov_drug_list.append(vdata['cov_drug'])
            noise = make_noise(b_size, 20)
            x_indices = vdata['top_gene_indices']
            x_raw = vdata['control_raw']
            gene_reconstructions , emb = model.to(device)(gene_token,x_indices,x_raw,control,encode_label, noise)
            dim = (gene_reconstructions.size(1)) // 3
            gene_means = gene_reconstructions[:, :dim]
            gene_vars = gene_reconstructions[:, dim:2*dim]
            gene_vars = F.softplus(gene_vars)
            gene_zi_logits = gene_reconstructions[:, 2*dim:]  
            zi_prob = torch.sigmoid(gene_zi_logits) 
            log_normal_dist = Normal(
                                    loc=torch.clamp(gene_means, min=1e-3, max=1e3),
                                    scale=torch.clamp(torch.sqrt(gene_vars), min=1e-3, max=1e3)
                                )   
            nb_sample = sample_ziln(
                            zi_prob=zi_prob,
                            log_normal_dist=log_normal_dist,
                            device=gene_means.device,  
                            truncate_max=1e4 
                        ).cpu().numpy()
            emb_list.append(emb)        
            control = vdata['control_raw']
            raw_control_list.append(control.cpu())
            raw_target_list.append(target.cpu())
            celltype = vdata['celltype']
            celltype_list.append(celltype)

                
            raw_control = control.cpu().numpy()
            encode_label = vdata['label']
          
            batch_samples.append(nb_sample)
            
            y_true = target.cpu().detach().numpy()

            
            loop_v.set_description(
                f"Iter {iteration+1}/{3} | "
            )
        
  

In [ ]:
all_control_original = np.vstack(raw_control_list)
all_target_original = np.vstack(raw_target_list)
all_predict_original = np.vstack(batch_samples)
cov_drug_list_original = np.concatenate(cov_drug_list,axis=0)